In [1]:
from sqlalchemy import create_engine
import pandas as pd
import sqlite3 as sql
import seaborn as sns
import numpy as np
import re
import matplotlib.pyplot as plt
import datetime

%matplotlib inline

In [2]:
police_calls = pd.read_excel('../data/police_calls.xlsx')

In [3]:
police_calls.head()

,Precinct,Date,Created,Address,Incident Type,Priority
0,C,07-01-2021,01:57:16,639 LAFAYETTE ST,INTOXICATED PERSON,2
1,C,07-01-2021,03:02:52,920 BROADWAY,WELFARE CHECK,2
2,C,07-01-2021,05:44:47,110 4TH AVE S,DISORDERLY SUBJECT,2
3,C,07-01-2021,05:49:07,800 4TH AVE S,SUSPICIOUS PERSON,2
4,C,07-01-2021,08:01:06,1033 DEMONBREUN ST,TRESPASSER,2


In [4]:
police_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48023 entries, 0 to 48022
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Precinct       48023 non-null  object
 1   Date           47926 non-null  object
 2   Created        47926 non-null  object
 3   Address        47926 non-null  object
 4   Incident Type  47926 non-null  object
 5   Priority       47926 non-null  object
dtypes: object(6)
memory usage: 2.2+ MB


In [5]:
police_calls = police_calls.dropna()

In [6]:
police_calls.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47926 entries, 0 to 48019
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Precinct       47926 non-null  object
 1   Date           47926 non-null  object
 2   Created        47926 non-null  object
 3   Address        47926 non-null  object
 4   Incident Type  47926 non-null  object
 5   Priority       47926 non-null  object
dtypes: object(6)
memory usage: 2.6+ MB


In [7]:
police_calls["Precinct"].value_counts()

H       7957
S       7953
C       6870
MT      5973
N       5515
M       4867
E       4408
W       4258
CW       118
PCW        4
CMD1       2
CMD2       1
Name: Precinct, dtype: int64

In [8]:
police_calls['Incident Type'].value_counts()

DISORDERLY SUBJECT      15568
ASSIST EMS              10043
WELFARE CHECK            9545
TRESPASSER               4023
SUSPICIOUS PERSON        2797
DOMESTIC DISTURBANCE     2661
SUICIDAL PERSON          1725
MENTAL CONDITION         1266
INTOXICATED PERSON        298
Name: Incident Type, dtype: int64

In [9]:
conn = sql.connect('../data/census.sqlite')
cur = conn.cursor()

In [11]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")

In [12]:
available_tables = (cur.fetchall())
print(available_tables)

[('B01001',), ('B03002',), ('B11001',), ('B11002',), ('B15002',), ('B19001',), ('B19013',), ('B19301',), ('B25024',), ('B25075',), ('S1701',)]


In [13]:
age_sex = pd.read_sql("SELECT * FROM B01001;", conn)

In [14]:
age_sex.head()

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B01001_001E,2533,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B01001_001E,2955,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B01001_001E,4435,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B01001_001E,3397,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B01001_001E,4157,Estimate!!Total:


In [16]:
race = pd.read_sql("SELECT * FROM B03002;", conn)

In [17]:
household_type = pd.read_sql("SELECT * FROM B11001;", conn)
pop_household = pd.read_sql("SELECT * FROM B11002;", conn)
education = pd.read_sql("SELECT * FROM B15002;", conn)
income = pd.read_sql("SELECT * FROM B19001;", conn)
housing_value = pd.read_sql("SELECT * FROM B25075;", conn)
poverty_status = pd.read_sql("SELECT * FROM S1701;", conn)

In [20]:
income.head(2)

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B19001_001E,990,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B19001_001E,1221,Estimate!!Total:


In [19]:
census = ['age_sex', 'race', 'household_type','pop_household','education','income','housing_value']

In [21]:
## for loops to remove 'Estimate!!Total:!!' from census list
##for i in census:
##    for d in description

In [22]:
active_in_care = police_calls.loc[(police_calls.Precinct == 'H') | (police_calls.Precinct == 'N')]
active_in_care.head(2)

,Precinct,Date,Created,Address,Incident Type,Priority
1264,H,07-01-2021,00:54:10,2827 SMITH SPRINGS RD,DISORDERLY SUBJECT,2
1265,H,07-01-2021,01:58:23,1000 THOMPSON PL,SUSPICIOUS PERSON,2


In [24]:
police_calls.loc[(police_calls.Precinct != 'H') & (police_calls.Precinct != 'N')]

,Precinct,Date,Created,Address,Incident Type,Priority
0,C,07-01-2021,01:57:16,639 LAFAYETTE ST,INTOXICATED PERSON,2
1,C,07-01-2021,03:02:52,920 BROADWAY,WELFARE CHECK,2
2,C,07-01-2021,05:44:47,110 4TH AVE S,DISORDERLY SUBJECT,2
3,C,07-01-2021,05:49:07,800 4TH AVE S,SUSPICIOUS PERSON,2
4,C,07-01-2021,08:01:06,1033 DEMONBREUN ST,TRESPASSER,2
...,...,...,...,...,...,...
48015,W,03-31-2022,11:46:52,3811 HILLSBORO PIKE,TRESPASSER,2
48016,W,03-31-2022,12:24:06,7277 CHARLOTTE PIKE,SUICIDAL PERSON,2
48017,W,03-31-2022,17:59:25,MURPHY RD / 42ND AVE N,DISORDERLY SUBJECT,2
48018,W,03-31-2022,20:31:47,2828 KENWAY RD,SUSPICIOUS PERSON,2
